In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
import json

# 1. Carregar os dados do JSON
def load_data_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return pd.DataFrame(data)

# Substitua 'seu_arquivo.json' pelo caminho do seu arquivo JSON
file_path = 'dados.json'
df = load_data_from_json(file_path)


print("Primeiras linhas do DataFrame:")
print(df.head())

# 2. Pré-processamento dos dados
# Separar features e target
X = df.drop('target', axis=1).values  # Substitua 'target' pelo nome da sua coluna alvo
y = df['target'].values.reshape(-1, 1)

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Implementação da Rede Neural com Backpropagation
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Inicializar pesos com valores aleatórios pequenos
        self.weights1 = np.random.randn(input_size, hidden_size) * 0.01
        self.weights2 = np.random.randn(hidden_size, output_size) * 0.01
        self.bias1 = np.zeros((1, hidden_size))
        self.bias2 = np.zeros((1, output_size))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, X):
        self.hidden = self.sigmoid(np.dot(X, self.weights1) + self.bias1)
        self.output = self.sigmoid(np.dot(self.hidden, self.weights2) + self.bias2)
        return self.output
    
    def backward(self, X, y, output, learning_rate):
        # Calcular gradientes
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)
        
        hidden_error = output_delta.dot(self.weights2.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden)
        
        # Atualizar pesos e biases
        self.weights2 += self.hidden.T.dot(output_delta) * learning_rate
        self.bias2 += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        self.weights1 += X.T.dot(hidden_delta) * learning_rate
        self.bias1 += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate
    
    def train(self, X, y, epochs, learning_rate):
        loss_history = []
        for epoch in range(epochs):
            # Forward pass
            output = self.forward(X)
            
            # Backward pass
            self.backward(X, y, output, learning_rate)
            
            # Calcular e armazenar perda
            loss = np.mean(np.square(y - output))
            loss_history.append(loss)
            
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')
        
        return loss_history
    
    def predict(self, X):
        return np.round(self.forward(X))

# 4. Criar e treinar a rede neural
input_size = X_train.shape[1]
hidden_size = 4  # Número de neurônios na camada oculta
output_size = 1  # Para classificação binária

nn = NeuralNetwork(input_size, hidden_size, output_size)
loss_history = nn.train(X_train, y_train, epochs=1000, learning_rate=0.1)

# 5. Avaliar o modelo
y_pred_train = nn.predict(X_train)
y_pred_test = nn.predict(X_test)

# Para problemas de classificação
if len(np.unique(y)) <= 2:  # Classificação binária
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    print(f'\nAcurácia - Treino: {train_acc:.4f}, Teste: {test_acc:.4f}')
else:  # Para regressão
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    print(f'\nMSE - Treino: {train_mse:.4f}, Teste: {test_mse:.4f}')

# 6. Plotar resultados
plt.figure(figsize=(12, 5))

# Gráfico da perda durante o treinamento
plt.subplot(1, 2, 1)
plt.plot(loss_history)
plt.title('Perda durante o treinamento')
plt.xlabel('Época')
plt.ylabel('Perda')

# Gráfico de valores reais vs preditos (para teste)
plt.subplot(1, 2, 2)
plt.scatter(range(len(y_test)), y_test, color='blue', label='Real', alpha=0.5)
plt.scatter(range(len(y_test)), y_pred_test, color='red', label='Predito', alpha=0.5)
plt.title('Valores Reais vs Preditos (Teste)')
plt.xlabel('Amostra')
plt.ylabel('Valor')
plt.legend()

plt.tight_layout()
plt.show()